In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Cell 1: Installation & Setup
# IMPORTANT: Restart your kernel before running this cell if you see pyarrow errors!
import os

# Install libraries
# We force upgrade pyarrow to ensure compatibility
!git clone https://github.com/QingruZhang/AdaLoRA.git
!pip install ./AdaLoRA/loralib
!pip install transformers datasets evaluate accelerate scikit-learn bitsandbytes
!pip install --upgrade pyarrow 

print("Installation complete. If you get a pyarrow error below, RESTART KERNEL and run again.")

Cloning into 'AdaLoRA'...
remote: Enumerating objects: 1603, done.
remote: Counting objects: 100% (1603/1603), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 1603 (delta 818), reused 1601 (delta 816), pack-reused 0 (from 0)
Receiving objects: 100% (1603/1603), 4.20 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (818/818), done.
Processing ./AdaLoRA/loralib
  Preparing metadata (setup.py) ... done
  Created wheel for loralib: filename=loralib-0.1.0-py3-none-any.whl size=13657 sha256=19d790197d094d17ce672d65dcd3699f9dddfb63abf368a0a1ee43954d9a36fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-x9fpzspz/wheels/21/55/60/9a2650d6e6f04c298a199d836cbc5df341129aab52666fc7fe
Successfully built loralib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [3]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [4]:
!pip install datasets
!pip install transformers
!pip install -U peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 10.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 179.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 218.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 285.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 340.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 342.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, AdaLoraConfig, TaskType  # <-- AdaLoRA here

os.environ["WANDB_DISABLED"] = "true"


2025-12-10 18:01:50.919773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765389711.138472      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765389711.202975      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TASK = "rte"
LR = 2e-4
NUM_EPOCHS = 5
BATCH_SIZE = 8
MAX_LEN = 256

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥 Using device: {device}")

🖥 Using device: cuda


In [7]:
print("📚 Loading dataset and tokenizer...")
dataset = load_dataset("glue", TASK)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

📚 Loading dataset and tokenizer...


Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [8]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

def clean_text(s):
    if s is None:
        return ""
    return " ".join(str(s).strip().split())

def preprocess_function(batch):
    sent1 = [clean_text(s) for s in batch["sentence1"]]
    sent2 = [clean_text(s) for s in batch["sentence2"]]
    return tokenizer(
        sent1,
        sent2,
        truncation=True,
        max_length=MAX_LEN,
    )

print("🔧 Tokenizing...")
encoded_dataset = dataset.map(preprocess_function, batched=True)

🔧 Tokenizing...


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [9]:
drop_cols = [
    col for col in ["sentence1", "sentence2", "idx"]
    if col in encoded_dataset["train"].column_names
]
if drop_cols:
    encoded_dataset = encoded_dataset.remove_columns(drop_cols)

if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

encoded_dataset.set_format("torch")

train_size = len(encoded_dataset["train"])
steps_per_epoch = (train_size + BATCH_SIZE - 1) // BATCH_SIZE
total_steps = steps_per_epoch * NUM_EPOCHS

print(f"✅ Train examples: {train_size}")
print(f"✅ Steps per epoch: {steps_per_epoch}")
print(f"✅ Total training steps: {total_steps}")

✅ Train examples: 2490
✅ Steps per epoch: 312
✅ Total training steps: 1560


In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)
print("🧠 Loading base model...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True,
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.to(device)

`torch_dtype` is deprecated! Use `dtype` instead!


🧠 Loading base model...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1

In [11]:
print("⚙️ Setting up AdaLoRA...")
peft_config = AdaLoraConfig(
    task_type=TaskType.SEQ_CLS,

    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],

    init_r=12,
    target_r=8,
    tinit=int(0.1 * total_steps),
    tfinal=int(0.8 * total_steps),
    deltaT=max(1, int(0.04 * total_steps)),
    beta1=0.85,
    beta2=0.85,
    orth_reg_weight=1e-3,
    total_step=total_steps,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


⚙️ Setting up AdaLoRA...
trainable params: 9,467,704 || all params: 1,043,984,338 || trainable%: 0.9069


In [12]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
print("🧾 Setting up Trainer...")
training_args = TrainingArguments(
    output_dir=f"{MODEL_ID}-RTE-adalora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)



🧾 Setting up Trainer...


In [14]:
print("🚀 Starting Training...")
trainer.train()

🚀 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy
1,1.539800,1.224081,0.664260
2,0.883600,1.064079,0.747292
3,0.819500,1.007446,0.768953
4,0.450100,1.108698,0.783394
5,0.388100,1.129738,0.790614


TrainOutput(global_step=780, training_loss=0.8699156149839744, metrics={'train_runtime': 2959.5025, 'train_samples_per_second': 4.207, 'train_steps_per_second': 0.264, 'total_flos': 1.3882996036834344e+16, 'train_loss': 0.8699156149839744, 'epoch': 5.0})

In [15]:
print("📊 Final Evaluation:")
results = trainer.evaluate()
print(results)

📊 Final Evaluation:


{'eval_loss': 1.129738211631775, 'eval_accuracy': 0.7906137184115524, 'eval_runtime': 29.9327, 'eval_samples_per_second': 9.254, 'eval_steps_per_second': 0.601, 'epoch': 5.0}
